In [ ]:
%matplotlib inline
from IPython.display import SVG, Image

import os.path
import sys
import re
import itertools
import csv
import datetime
import pickle
import random
from collections import defaultdict, Counter
import gc

import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
import seaborn as sns
import pandas as pd
import numpy as np
import scipy
import gensim
from sklearn.metrics import f1_score, classification_report, confusion_matrix, log_loss
from sklearn.model_selection import train_test_split
import gensim
from keras.preprocessing.sequence import skipgrams
import tensorflow as tf

In [ ]:
img_cnt = 0
def save_img():
    global img_cnt
    img_cnt += 1
    img_name = 'img_{0:03}.jpeg'.format(img_cnt)
    print(img_name)
    plt.savefig(img_name, pil_kwargs={'quality': 100})

In [ ]:
from numpy.random import default_rng
rng = default_rng(10)

def make_mat(nrow, ncol, from_row, to_row, ratio):
    nn = int(ncol*ratio)
    idx = np.arange(ncol)
    ret0 = np.zeros((nrow, ncol), dtype=int)
    for ii in np.arange(from_row, to_row):
        ret0[ii, rng.choice(idx, nn, replace=False)] = 1
    return ret0

# Create Data
---

In [ ]:
N_ROW = 250

In [ ]:
mat1 = make_mat(N_ROW, 5, 0, 50, 0.6)
mat1

In [ ]:
mat1_1 = make_mat(N_ROW, 7, 0, 100, 0.6)
mat1_1

In [ ]:
plt.figure(figsize=(20, 20))
plt.imshow(np.c_[mat1, mat1_1][:600,:].T)

In [ ]:
mat2 = make_mat(N_ROW, 9, 20, 100, 0.6)
mat2

In [ ]:
plt.figure(figsize=(20, 20))
plt.imshow(np.c_[mat1, mat1_1, mat2][:600,:].T)

In [ ]:
mat3 = make_mat(N_ROW, 10, 20, 150, 0.6)
mat3

In [ ]:
plt.figure(figsize=(20, 20))
plt.imshow(np.c_[mat1, mat1_1, mat2, mat3][:700,:].T)

In [ ]:
mat4 = make_mat(N_ROW, 7, 80, 150, 0.6)
mat4

In [ ]:
plt.figure(figsize=(20, 20))
plt.imshow(np.c_[mat1, mat1_1, mat2, mat3, mat4][:700,:].T)

In [ ]:
mat5 = make_mat(N_ROW, 10, 80, 200, 0.6)
mat5

In [ ]:
plt.figure(figsize=(20, 20))
plt.imshow(np.c_[mat1, mat1_1, mat2, mat3, mat4, mat5].T)

In [ ]:
mat6 = make_mat(N_ROW, 5, 120, 200, 0.6)
mat6

In [ ]:
plt.figure(figsize=(20, 20))
plt.imshow(np.c_[mat1, mat1_1, mat2, mat3, mat4, mat5, mat6].T)

In [ ]:
mat7 = make_mat(N_ROW, 5, 180, 250, 0.6)
mat7

In [ ]:
plt.figure(figsize=(20, 20))
plt.imshow(np.c_[mat1, mat1_1, mat2, mat3, mat4, mat5, mat6, mat7].T)

In [ ]:
X = np.c_[mat1, mat1_1, mat2, mat3, mat4, mat5, mat6, mat7]
X.shape

In [ ]:
X_df = pd.DataFrame(X, dtype=int)
X_df.index = ['r'+ee.astype('str') for ee in (np.arange(X_df.shape[0])+1)]
X_df.columns = ['c'+ee.astype('str') for ee in np.arange(X_df.shape[1])+1]
#X_df['cls'] = ['cls'+str(ii) for ii in cls_user]
print(X_df.shape)
X_df.head()

In [ ]:
X_df.to_csv('sample_data.csv')

In [ ]:
plt.figure(figsize=(20, 20))
plt.imshow(X.T)
save_img()

In [ ]:
from numpy.random import default_rng
rng = default_rng(10)
idx = np.arange(X_df.shape[0])
rng.shuffle(idx)

plt.figure(figsize=(20, 20))
plt.imshow(X[idx,:].T)
save_img()

# 元のデータに因子分析と主成分分析
---
* [apply factanal to sample014](https://www.kaggle.com/wordroid/apply-factanal-to-sample014)

```
Loadings:
    Factor1 Factor2 Factor3 Factor4 Factor5 Factor6 Factor7 Factor8
c1          -0.230  -0.117   0.672                                 
c2   0.155  -0.224  -0.117   0.611                                 
c3   0.139  -0.205  -0.123   0.576                                 
c4          -0.208  -0.110   0.608                                 
c5          -0.200  -0.107   0.573                                 
c6   0.503  -0.249           0.400           0.145   0.228         
c7   0.456  -0.210           0.337                   0.126         
c8   0.491  -0.243           0.399                           0.112 
c9   0.387  -0.213           0.330           0.225  -0.182   0.146 
c10  0.495  -0.246           0.189  -0.121   0.139                 
c11  0.537  -0.209           0.266  -0.109                   0.107 
c12  0.449  -0.258           0.252                  -0.155  -0.155 
c13  0.592  -0.194           0.121           0.122  -0.142         
c14  0.678  -0.161           0.113                   0.271   0.112 
c15  0.629  -0.184                          -0.108                 
c16  0.508  -0.135                  -0.143   0.223   0.192   0.186 
c17  0.539  -0.167           0.136           0.808                 
c18  0.636  -0.182                                  -0.111         
c19  0.666  -0.158                                   0.135         
c20  0.557  -0.186           0.179                                 
c21  0.560  -0.154                           0.130   0.132         
c22  0.432           0.381   0.120   0.131   0.182   0.241         
c23  0.433           0.279   0.123   0.108           0.517         
c24  0.462           0.451           0.124                  -0.152 
c25  0.459           0.332           0.165                         
c26  0.363           0.382   0.153           0.275   0.235         
c27  0.334           0.321   0.109   0.284                   0.265 
c28  0.527           0.349           0.211          -0.118         
c29  0.422           0.422           0.162                   0.189 
c30  0.471           0.395           0.250                         
c31  0.499           0.223           0.370                  -0.101 
c32          0.151   0.614                                         
c33          0.217   0.491           0.727                   0.134 
c34          0.177   0.678                                         
c35          0.167   0.700                                         
c36          0.199   0.690           0.198                   0.242 
c37          0.287   0.472           0.423                         
c38  0.186   0.208   0.363           0.125          -0.147   0.372 
c39          0.568   0.203  -0.140   0.325                   0.105 
c40          0.547   0.342  -0.114                           0.101 
c41          0.584   0.307  -0.138                  -0.107   0.124 
c42          0.448   0.307  -0.113   0.110                         
c43          0.523   0.307  -0.142  -0.149                   0.195 
c44          0.458   0.444  -0.122                           0.137 
c45          0.527   0.180  -0.122                   0.128  -0.104 
c46 -0.126   0.548   0.239  -0.101   0.198                         
c47          0.547   0.276  -0.150                                 
c48          0.571   0.158  -0.126   0.199                         
c49 -0.183   0.708                                          -0.273 
c50 -0.177   0.654  -0.195  -0.100                                 
c51 -0.175   0.585                                          -0.206 
c52 -0.178   0.588                                           0.141 
c53 -0.175   0.483          -0.102                                 
c54 -0.383  -0.191  -0.306  -0.361  -0.104                   0.117 
c55 -0.395  -0.210  -0.302  -0.369  -0.101                         
c56 -0.384  -0.217  -0.260  -0.352  -0.111                         
c57 -0.381  -0.308  -0.217  -0.341                                 
c58 -0.356  -0.112  -0.296  -0.332  -0.121                         

               Factor1 Factor2 Factor3 Factor4 Factor5 Factor6 Factor7 Factor8
SS loadings      7.837   6.101   5.043   3.604   1.609   1.040   0.857   0.823
Proportion Var   0.135   0.105   0.087   0.062   0.028   0.018   0.015   0.014
Cumulative Var   0.135   0.240   0.327   0.389   0.417   0.435   0.450   0.464


Loadings:
    MR4    MR3    MR1    MR2    MR5    MR7    MR8    MR6   
c1          0.703                                          
c2          0.627  0.146                      -0.145       
c3          0.598  0.136 -0.131               -0.199       
c4          0.637 -0.113               -0.109  0.167       
c5          0.567                                    -0.110
c6          0.328  0.451        -0.120                     
c7          0.275  0.308                0.103         0.103
c8  -0.115  0.341  0.102 -0.108         0.292         0.193
c9  -0.113  0.269               -0.205         0.466  0.160
c10 -0.136         0.107        -0.251  0.347  0.145       
c11         0.200  0.294 -0.134         0.152         0.175
c12 -0.103  0.156               -0.128  0.626              
c13                                     0.552         0.132
c14                0.567 -0.168         0.125         0.155
c15                0.183 -0.254         0.258  0.378       
c16                0.485        -0.245                0.259
c17                0.344        -0.254  0.185  0.190       
c18                0.162 -0.154         0.410  0.193  0.118
c19        -0.108  0.424 -0.124 -0.113  0.182  0.194  0.149
c20                0.329        -0.163         0.370       
c21                0.365 -0.147         0.343              
c22                0.451  0.228  0.143         0.117       
c23                0.683  0.115  0.166               -0.135
c24        -0.109  0.216  0.263  0.183  0.467              
c25                              0.170         0.598       
c26                0.451  0.281                0.211       
c27         0.114  0.105         0.366         0.171  0.347
c28        -0.146                0.247  0.456  0.140  0.106
c29                0.112  0.187  0.195  0.220         0.320
c30                0.108  0.102  0.288  0.157  0.373       
c31                0.170 -0.113  0.375  0.289  0.291       
c32                       0.515  0.194                     
c33                              0.799                     
c34                       0.564  0.200                     
c35                       0.686                            
c36                       0.485  0.326                0.230
c37  0.130                0.158  0.576         0.140       
c38                       0.136  0.175         0.127  0.481
c39  0.458                       0.301        -0.115  0.142
c40  0.422                0.332                       0.190
c41  0.460        -0.109  0.163  0.132                0.173
c42  0.330                0.244  0.138        -0.107       
c43  0.413 -0.117         0.303 -0.139                0.216
c44  0.306                0.430               -0.142  0.230
c45  0.496 -0.107  0.133  0.221                            
c46  0.448                0.108  0.215                     
c47  0.470 -0.137         0.205                            
c48  0.493                       0.241 -0.141              
c49  0.730                                           -0.188
c50  0.688               -0.196                            
c51  0.620                                           -0.184
c52  0.570               -0.177        -0.122         0.176
c53  0.434                       0.128 -0.132              
c54 -0.249 -0.323        -0.186        -0.263 -0.184       
c55 -0.271 -0.331        -0.182        -0.280 -0.179       
c56 -0.276 -0.323        -0.110 -0.114 -0.249 -0.119       
c57 -0.381 -0.314 -0.117               -0.230        -0.118
c58 -0.154 -0.296        -0.174 -0.122 -0.248 -0.186       

                 MR4   MR3   MR1   MR2   MR5   MR7   MR8   MR6
SS loadings    4.361 3.154 2.731 2.637 2.453 2.416 1.690 1.192
Proportion Var 0.075 0.054 0.047 0.045 0.042 0.042 0.029 0.021
Cumulative Var 0.075 0.130 0.177 0.222 0.264 0.306 0.335 0.356
```

In [ ]:
from sklearn import decomposition
pca = decomposition.PCA(n_components=2)

In [ ]:
pca.fit(X_df)
rscore = pca.transform(X)
print(rscore.shape)

In [ ]:
df = pd.DataFrame(rscore[:,:5])
sns.pairplot(df, markers='o', height=3.5, diag_kind='hist')
save_img()

In [ ]:
df = pd.DataFrame(pca.components_.T[:,:5])
sns.pairplot(df, markers='o', height=3.5, diag_kind='hist')
save_img()

In [ ]:
from sklearn import decomposition
fa = decomposition.FactorAnalysis(n_components=8)

In [ ]:
fa.fit(X_df)
rscore = fa.transform(X)
print(rscore.shape)

In [ ]:
pd.DataFrame(fa.components_.T)

In [ ]:
df = pd.DataFrame(rscore[:,:5])
sns.pairplot(df, markers='o', height=3.5, diag_kind='hist')
save_img()

In [ ]:
df = pd.DataFrame(fa.components_.T[:,:5])
sns.pairplot(df, markers='o', height=3.5, diag_kind='hist')
save_img()

# 特徴量を算出
---
* [lowcol sample014 fet](https://www.kaggle.com/wordroid/lowcol-sample014-fet)

In [ ]:
Image('../input/lowcol-sample014-fet/img_008.jpeg')

# プロット
---
* [lowcol sample014 plot](https://www.kaggle.com/wordroid/lowcol-sample014-plot)

In [ ]:
Image('../input/lowcol-sample014-plot/img_001.png')

In [ ]:
Image('../input/lowcol-sample014-plot/img_002.png')

# 相関（距離類似度）
---
* [lowcol sample014 cor](https://www.kaggle.com/wordroid/lowcol-sample014-cor)

# 因子分析
---
* [apply factanal to sample014 cor](https://www.kaggle.com/wordroid/apply-factanal-to-sample014-cor)
* ランク落ちする
* faを使う

```
'Mclust' model object: (VVI,8)


Loadings:
    MR1    MR2    MR8    MR3    MR4    MR5    MR7    MR6   
c1                        0.125         0.938              
c2                                      0.953              
c3                                      1.007              
c4                                      0.998              
c5                                      1.021              
c6                        0.911         0.115              
c7                        0.996                            
c8                        0.986                            
c9                        1.009                            
c10  0.115                0.961                            
c11                       0.968                            
c12                       1.005                            
c13                0.999                                   
c14  0.192         0.872                                   
c15                1.041                                   
c16                0.983                                   
c17                0.941                                   
c18                1.015                                   
c19  0.108         0.935                                   
c20                0.963                                   
c21 -0.104         1.045                                   
c22  0.961                                                 
c23  0.953                                                 
c24  0.970                                                 
c25  0.987                                                 
c26  0.982                                                 
c27  0.946        -0.140  0.192                            
c28  0.955         0.103                                   
c29  0.999               -0.103                            
c30  0.995                                                 
c31  0.932         0.103                                   
c32                              1.013                     
c33                              0.884         0.107       
c34                              0.998                     
c35                              0.966                     
c36  0.151                       0.941                     
c37                              0.840         0.140       
c38                              0.896                     
c39         0.850                              0.217       
c40         0.979                                          
c41         0.830                              0.229       
c42         1.041                                          
c43         1.036                                          
c44         1.037                             -0.126       
c45         0.998                                          
c46         0.743                              0.343       
c47         1.014                                          
c48         0.817                              0.266       
c49         0.386                              0.720       
c50                                            0.935       
c51                                            1.026       
c52                                            0.977       
c53                                            0.951       
c54                                                   0.963
c55                                                   0.986
c56                                                   0.869
c57                                                   0.664
c58                                                   0.924

                 MR1   MR2   MR8   MR3   MR4   MR5   MR7   MR6
SS loadings    9.539 9.039 8.689 6.799 6.165 4.865 4.677 3.957
Proportion Var 0.164 0.156 0.150 0.117 0.106 0.084 0.081 0.068
Cumulative Var 0.164 0.320 0.470 0.587 0.694 0.778 0.858 0.926
```